In [ ]:
# import functions from .py files
import pprint, sqlite3
from SPASE_Scraper_Script import SPASE_Scraper
from PathGrabber import getPaths

# list that holds paths returned by PathGrabber
SPASE_paths = []

# print Flag (True prints, False does not)
printFlag = False

# get user input and extract all SPASE records
print("Enter root folder you want to search")
folder = input()
print("You entered " + folder)
SPASE_paths = getPaths(folder, SPASE_paths)
if printFlag:
    print("The number of records is "+ str(len(SPASE_paths)))
    print("The SPASE records found are:")
    print(SPASE_paths)
print("===========================================================================================================")

# lists that hold URLs to be exported and SPASE records already checked
lines = []
searched = []

# iterate through all SPASE records returned by PathGrabber
for record in SPASE_paths:
    # scrape metadata for each record
    if record not in searched:
        (RID, RIDField, author, authorField, pub, pubField, pubDate, pubDateField, dataset, datasetField, 
         desc, descField, PI, PIField, AccessRights, licenseField, datalinkField) = SPASE_Scraper(record)
        searched.append(record)
        pubYear = pubDate[0:4]
        author = ", ".join(author)
        if printFlag:
            print("The ResourceID is " + RID + " which was obtained from " + RIDField)
            print("The author(s) are " + author + " which was obtained from " + authorField)
            print("The publication year is " + pubYear + " which was obtained from " + pubDateField)
            print("The publisher is " + pub + " which was obtained from " + pubField)
            print("The dataset is " + dataset + " which was obtained from " + datasetField)
            print("The description is " + desc + " which was obtained from " + descField)
            print("The persistent identifier is " + PI + " which was obtained from " + PIField)
            print("The URLs with their associated product keys obtained from " + datalinkField + """ and their 
                  license(s) obtained from """ + licenseField + " are: ")
            pprint.pprint(AccessRights)

        datalink = ""
        license = ""
        desired = False

        # separate license from the datalink and product keys from AccessRights to store in db
        for k, v in AccessRights.items():
            if not v:
                continue
            else:
                license = k
                # check if any urls are for consideration
                for link in v.keys():
                    if ("nasa.gov" or "virtualsolar.org") in link:
                        desired = True
                #datalink = str(v)
                for key, val in v.items():
                    # allow desired URLs to be added to database
                    if ("nasa.gov" or "virtualsolar.org") in key:
                        datalink += str(key) + ": " + str(val) + " \n"
                    else:
                        # if no desired URLs present, add URLs to text file list
                        if not desired:
                            lines.append(RID + ", " + str(key))
                if printFlag:
                    print(k + " was assigned to license")
                    #for key, val in v.items():
                     #   datalink = key
                      #  prodKey = val
                    print(datalink + " was assigned to datalink")


        # add table to existing database

        def create_tables():
            sql_statements = [ 
                """CREATE TABLE IF NOT EXISTS SPASE_Metadata (
                        id INTEGER PRIMARY KEY, 
                        SPASE_id TEXT NOT NULL UNIQUE, 
                        author TEXT,
                        author_source TEXT,
                        publisher TEXT,
                        publisher_source TEXT,
                        publication_yr TEXT,
                        publication_yr_source TEXT,
                        dataset TEXT,
                        dataset_source TEXT,
                        license TEXT,
                        license_source TEXT,
                        datalink TEXT,
                        datalink_source TEXT,
                        description TEXT,
                        description_source TEXT,
                        PI TEXT,
                        PI_source TEXT
                );"""]

            # create a database connection
            try:
                with sqlite3.connect('SPASE_Data.db') as conn:
                    cursor = conn.cursor()
                    for statement in sql_statements:
                        cursor.execute(statement)

                    conn.commit()
            except sqlite3.Error as e:
                print(e)

        create_tables()

        # insert metadata entries into table

        def add_SPASE(conn, entry):
            sql = '''INSERT INTO SPASE_Metadata(SPASE_id,author,author_source,publisher,publisher_source,
                    publication_yr,publication_yr_source,dataset,dataset_source,license,license_source,
                    datalink,datalink_source,description,description_source,PI,PI_source)
                    VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) '''
            cur = conn.cursor()
            cur.execute(sql, entry)
            conn.commit()
            return cur.lastrowid


        try:
            with sqlite3.connect('SPASE_Data.db') as conn:
                # add a new SPASE Record
                SPASE_Record = (RID, author, authorField, pub, pubField, pubYear, pubDateField, 
                                dataset, datasetField, license, licenseField, datalink, datalinkField,
                                desc, descField, PI, PIField)
                Record_id = add_SPASE(conn, SPASE_Record)
                print(f'Created a SPASE Record with the id {Record_id}')

        except sqlite3.Error as e:
            print(e)

        print("===========================================================================================================")

    else:
        continue
        
# add SPASE records without desired links to a text file
textFile = open("../NoLinkRecords.txt", "a")
repeat = []
for line in lines:
    rid, url = line.split(",")
    if url not in repeat:
        textFile.write(line)
        textFile.write("\n")
        repeat.append(url)
textFile.close()

# call .py file directly from notebook
#%run ./SPASE_test.py {path} in notebook
#import sys
#args = sys.argv
#path = args[1] in source file

# test path : Big = "/home/jovyan/NASA/NumericalData/DE2" or Small = "/home/jovyan/NASA/NumericalData/ACE/EPAM/RATES"

In [3]:
textFile = open("../NoLinkRecords.txt", "w")
textFile.close()

In [10]:
from SPASE_Scraper_Script import SPASE_Scraper

SPASE_Scraper.__doc__

'Takes path of a .xml SPASE record file and returns a tuple of values of varying types which hold all \n    desired metadata and the fields they came from. This will collect the desired metadata following the \n    priority rules determined by SPASE record experts. If any desired metadata is not found, the default \n    value assigned is an empty string.\n    \n    :param path: A string of the absolute/relative path of the SPASE record to be scraped.\n    :type path: String\n    :return: A tuple containing the metadata desired and where they were obtained.\n    :rtype: tuple\n    '